# WORKSHOP KERAS

Pour ce workshop nous allons créer pas à pas un [réseau de neurones récurrent](https://fr.wikipedia.org/wiki/R%C3%A9seau_de_neurones_r%C3%A9currents) (RNN) à l'aide de la librairie Keras. Ce réseau nous permettra de prédire le prix des actions Google.

Keras est une librairie qui permet de faire du deep learning en python, elle est basée sur une autre librairie développé par Google, [Tensorflow](https://www.tensorflow.org/).

## Les imports des librairies

Nous allons en premier lieu importer les librairies nécessaires pour ce workshop.

Voici les imports que nous allons utiliser pour Keras:

In [ ]:
from keras.models import Sequential       # use for NN
from keras.layers import Dense            # use for NN
from keras.layers import LSTM             # use for RNN
from keras.layers import Dropout          # use for NN

Ensuite, nous avons les imports des outils nécessaires au traitement des données:

In [ ]:
import numpy as np                               # use for array management
import matplotlib.pyplot as plt                  # use for display data
import pandas as pd                              # use for get csv data
from sklearn.preprocessing import MinMaxScaler   # use for data processing

maintenant que tous nos outils sont prêts nous allons commencer le traitement de données.

## Récupération et Traitement des données

Nous allons récupérer les données des CSV à l'aide de pandas (les csv sont [ici](https://bit.ly/2wnGlkg) et [ici](https://bit.ly/2QseoRw)).

In [ ]:
# open the train dataset
dataset_train = pd.read_csv("Google_Stock_Price_Train.csv")
train_set = dataset_train[["Open"]].values

# open the test dataset
dataset_test = pd.read_csv("Google_Stock_Price_Test.csv")
real_stock_price = dataset_test.iloc[:, 1:2].values

# data processing for scaling data in [0, 1] interval
sc = MinMaxScaler(feature_range=(0,1))
train_set_scaled = sc.fit_transform(train_set)

#divide data into xt (use for predict) and yt (the prediction result)
xt = []
yt = []

for i in range(60, 1238):
	xt.append(train_set_scaled[i-60:i, 0])
	yt.append(train_set_scaled[i, 0])
xt = np.array(xt)
yt = np.array(yt)

xt = np.reshape(xt, (xt.shape[0], xt.shape[1], 1))

Nos données sont correctement traitées, nous allons pouvoir les exploiter.

## Création du réseau de neurones

Nous allons ici créer notre réseau de neurones à l'aide de Keras, voici le détail des différentes fonctions utilisées:

| Fonction | détails |
|: ----------|: -------|
|Sequential()|creation de la base du réseau de neurones|
|.add(LSTM...)|ajout d'une couche de neurones de type LSTM|
|.add(Dropout())|probabilité qu'un neurone soit désactivé pour améliorer l'entrainement|
|.add(Dense())|création d'une couche de densité x de neurones|
|.compile()|compilation du réseau de neurones|
|.fit()|entrainement du réseau de neurones|

Vous pouvez modifier les valeurs pour arriver à un résultat plus pertinent.

In [ ]:
print("start_training :")

regressor = Sequential()

# Units is the dimension of the inner cells
# Return sequences defines if we can see the result of the hidden layers
# Input shape defines the template of the input
regressor.add(LSTM(units=45, return_sequences=True, input_shape=(xt.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=45, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=45, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.21))

# This layer is the final layer, it MUST be 1 because we only wait 1 output
regressor.add(Dense(units=1))

# During the compilation, the optimize and the loss define how the neural network is train.
regressor.compile(optimizer="adam", loss="mean_squared_error")

# epochs define the number of training cicles
# batch size is the number of inputs by epochs 
regressor.fit(xt, yt, epochs=50, batch_size=32)

print("Created !")

Maintenant que le réseau de neuronne est entrainé, nous pouvons le tester avec nos données pour voir le résultat.

## Prediction des actions de Google

Maintenant nous allons utiliser le modèle entrainé précédemment pour prédire le prix des actions de Google du lendemain.

### traitement des données de prédictions

Nous commencerons par traiter les données pour pouvoir les utiliser lors de la prédiction.

In [ ]:
# make the total dataset for run the rnn
dataset_total = pd.concat((dataset_train["Open"], dataset_test["Open"]), axis=0)

# scale data for the prediction
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)

xtt = []

# fill the prediction array
for i in range(61, 101):
    xtt.append(inputs[i-60:i, 0])
xtt = np.array(xtt)

xtt = np.reshape(xtt, (xtt.shape[0], xtt.shape[1], 1))

### Prédiction

Maintenant que tout est fin prêt nous allons pouvoir passer à la prédiction.

In [ ]:
# prediction of the stock price with .predict() function from Keras
predict = regressor.predict(xtt)

# transform the scaled data to the readable data
predict = sc.inverse_transform(predict)

print(real_stock_price)
print(predict)

Il ne reste plus qu'a afficher les resultats sous forme de schéma.

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predict, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

Voila ! Nous avons créé et entrainé notre réseau de neurone à l'aide de keras.

Pour plus d'information sur Keras vous pouvez jeter un coup d'oeil [ici](https://keras.io/).